# Multiome with Torch
This notebooks is to help competitors that would like to apply Deep Neural Networks to the MSCI data.
It is focused on the more challenging Multiome part of the data (but it is trivial to adapt it to to the CITEseq data)

The main challenge here is that the Multiome data is very large while Kaggle  GPU machines only have 13GB RAM + 16GB GPU Memory.

I found it is actually possible to store all of the dataset in GPU memory using sparse tensor formats. This uses ~12GB on the GPU, leaving only ~4GB for the model parameters and the forward/backward computation. Given that we have only ~100K training examples, I do not expect we will need very large models, so I feel 4GB is actually enough.

If 4GB is not enough, the other option is to leave the dataset in RAM and load the batches on demand to the GPU (which is what is more classically done). In that case, however, we have only ~1GB RAM left, and will suffer a small performance penalty from having to load the batches to the GPU. But we will have the whole 16GB available for training a complex model. Yet another option is to apply dimensionality reduction to the data beforehand (e.g. with PCA/TruncatedSVD), although I like more the idea of using the raw data and letting the network do its own dimensionality reduction.

The competition data is pre-encoded as sparse matrices in [this dataset](https://www.kaggle.com/datasets/fabiencrom/multimodal-single-cell-as-sparse-matrix) generated by [this notebook](https://www.kaggle.com/code/fabiencrom/multimodal-single-cell-creating-sparse-data/).

The model used here is just a very simple MLP. In the current version, I add a `Softplus` activation at the end, considering the values we have to predict are all positives (although I am not sure it will really work better that way).

In the current version, I also directly optimize the competition metric (row-wise Pearson correlation). Although it does not seem to perform much better than using a simpler Mean Square Error Loss.

This notebook will train 5 models over 5 folds. The final submission is created in [this notebook](https://www.kaggle.com/fabiencrom/msci-multiome-torch-quickstart-submission).

So far I did not get results better than the one obtained by the much simpler PCA+Ridge Regression method (that you can find in [this notebook](https://www.kaggle.com/code/ambrosm/msci-multiome-quickstart) as initially proposed by AmbrosM or in [this notebook](https://www.kaggle.com/code/fabiencrom/msci-multiome-quickstart-w-sparse-matrices) for a version using sparse matrices for better results). But I expect it will perform better after working on the architecture/hyperparameters. In any case, I think a deep learning model will be a part of any winning submission.

In [1]:
import os
import copy
import gc
import math
import itertools
import pickle
import glob
import joblib
import json
import random
import re
import operator

import collections
from collections import defaultdict
from operator import itemgetter, attrgetter

from tqdm.notebook import tqdm

import torch
import torch.nn as nn

import numpy as np
import pandas as pd
import plotly.express as px

import scipy

import sklearn
import sklearn.cluster
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import sklearn.preprocessing

import copy


# Score and loss functions
We can use either a classic Mean Square Error loss (nn.MSELoss) or use a loss that will optimize directly the competition metric.

In [2]:
def partial_correlation_score_torch_faster(y_true, y_pred):
    """Compute the correlation between each rows of the y_true and y_pred tensors.
    Compatible with backpropagation.
    """
    y_true_centered = y_true - torch.mean(y_true, dim=1)[:,None]
    y_pred_centered = y_pred - torch.mean(y_pred, dim=1)[:,None]
    cov_tp = torch.sum(y_true_centered*y_pred_centered, dim=1)/(y_true.shape[1]-1)
    var_t = torch.sum(y_true_centered**2, dim=1)/(y_true.shape[1]-1)
    var_p = torch.sum(y_pred_centered**2, dim=1)/(y_true.shape[1]-1)
    return cov_tp/torch.sqrt(var_t*var_p)

def correl_loss(pred, tgt):
    """Loss for directly optimizing the correlation.
    """
    return -torch.mean(partial_correlation_score_torch_faster(tgt, pred))

# Config
We put the configuration dict at the beginning of the notebook, so that it is easier to find and modify

In [3]:
config = dict(
    layers = [128, 128, 128],
    patience = 4,
    max_epochs = 20,
    criterion = correl_loss, #nn.MSELoss(),
    
    n_folds = 5,
    folds_to_train = [0, 1, 2, 3, 4],
    kfold_random_state = 42,
    
    optimizerparams = dict(
     lr=1e-3, 
     weight_decay=1e-2
    ),
    
    head="softplus"
    
)

INPUT_SIZE = 228942
OUTPUT_SIZE = 23418


# Utility functions for loading and batching the sparse data in device memory
There are a few challenges here:
- If we directly try to create a torch sparse tensor before moving it to memory, we will get an OOM error
- Torch CSR tensors cannot be moved to the gpu; so we make our own TorchCSR class that will contain the csr format information
- torch gpu operations are only compatible with COO tensors (not CSR), so we need some functions to create batches of COO tensors from the TorchCSR objects

In [4]:
# Strangely, current torch implementation of csr tensor do not accept to be moved to the gpu. 
# So we make our own equivalent class
TorchCSR = collections.namedtuple("TrochCSR", "data indices indptr shape")

def load_csr_data_to_gpu(train_inputs):
    """Move a scipy csr sparse matrix to the gpu as a TorchCSR object
    This try to manage memory efficiently by creating the tensors and moving them to the gpu one by one
    """
    th_data = torch.from_numpy(train_inputs.data).to(device)
    th_indices = torch.from_numpy(train_inputs.indices).to(device)
    th_indptr = torch.from_numpy(train_inputs.indptr).to(device)
    th_shape = train_inputs.shape
    return TorchCSR(th_data, th_indices, th_indptr, th_shape)

def make_coo_batch(torch_csr, indx):
    """Make a coo torch tensor from a TorchCSR object by taking the rows indicated by the indx tensor
    """
    th_data, th_indices, th_indptr, th_shape = torch_csr
    start_pts = th_indptr[indx]
    end_pts = th_indptr[indx+1]
    coo_data = torch.cat([th_data[start_pts[i]: end_pts[i]] for i in range(len(start_pts))], dim=0)
    coo_col = torch.cat([th_indices[start_pts[i]: end_pts[i]] for i in range(len(start_pts))], dim=0)
    coo_row = torch.repeat_interleave(torch.arange(indx.shape[0], device=device), th_indptr[indx+1] - th_indptr[indx])
    coo_batch = torch.sparse_coo_tensor(torch.vstack([coo_row, coo_col]), coo_data, [indx.shape[0], th_shape[1]])
    return coo_batch


def make_coo_batch_slice(torch_csr, start, end):
    """Make a coo torch tensor from a TorchCSR object by taking the rows within the (start, end) slice
    """
    th_data, th_indices, th_indptr, th_shape = torch_csr
    start_pts = th_indptr[start]
    end_pts = th_indptr[end]
    coo_data = th_data[start_pts: end_pts]
    coo_col = th_indices[start_pts: end_pts]
    coo_row = torch.repeat_interleave(torch.arange(end-start, device=device), th_indptr[start+1:end+1] - th_indptr[start:end])
    coo_batch = torch.sparse_coo_tensor(torch.vstack([coo_row, coo_col]), coo_data, [end-start, th_shape[1]])
    return coo_batch


# GPU memory DataLoader
We create a dataloader that will work with the in-device TorchCSR tensor.
This should ensure the fastest training speed.

In [5]:
class DataLoaderCOO:
    """Torch compatible DataLoader. Works with in-device TorchCSR tensors.
    Args:
         - train_inputs, train_targets: TorchCSR tensors
         - train_idx: tensor containing the indices of the rows of train_inputs and train_targets that should be used
         - batch_size, shuffle, drop_last: as in torch.utils.data.DataLoader
    """
    def __init__(self, train_inputs, train_targets, train_idx=None, 
                 *,
                batch_size=512, shuffle=False, drop_last=False):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.drop_last = drop_last
        
        self.train_inputs = train_inputs
        self.train_targets = train_targets
        
        self.train_idx = train_idx
        
        self.nb_examples = len(self.train_idx) if self.train_idx is not None else len(train_inputs)
        
        self.nb_batches = self.nb_examples//batch_size
        if not drop_last and not self.nb_examples%batch_size==0:
            self.nb_batches +=1
        
    def __iter__(self):
        if self.shuffle:
            shuffled_idx = torch.randperm(self.nb_examples, device=device)
            if self.train_idx is not None:
                idx_array = self.train_idx[shuffled_idx]
            else:
                idx_array = shuffled_idx
        else:
            if self.train_idx is not None:
                idx_array = self.train_idx
            else:
                idx_array = None
            
        for i in range(self.nb_batches):
            slc = slice(i*self.batch_size, (i+1)*self.batch_size)
            if idx_array is None:
                inp_batch = make_coo_batch_slice(self.train_inputs, i*self.batch_size, (i+1)*self.batch_size)
                tgt_batch = make_coo_batch_slice(self.train_targets, i*self.batch_size, (i+1)*self.batch_size)
            else:
                idx_batch = idx_array[slc]
                inp_batch = make_coo_batch(self.train_inputs, idx_batch)
                tgt_batch = make_coo_batch(self.train_targets, idx_batch)
            yield inp_batch, tgt_batch
            
            
    def __len__(self):
        return self.nb_batches

# Simple Model: MLP

In [6]:
class MLP(nn.Module):
    def __init__(self, layer_size_lst, add_final_activation=False):
        super().__init__()
        
        assert len(layer_size_lst) > 2
        
        layer_lst = []
        for i in range(len(layer_size_lst)-1):
            sz1 = layer_size_lst[i]
            sz2 = layer_size_lst[i+1]
            layer_lst += [nn.Linear(sz1, sz2)]
            if i != len(layer_size_lst)-2 or add_final_activation:
                 layer_lst += [nn.ReLU()]
        self.mlp = nn.Sequential(*layer_lst)
        
    def forward(self, x):
        return self.mlp(x)
    
def build_model():
    model = MLP([INPUT_SIZE] + config["layers"] + [OUTPUT_SIZE])
    if config["head"] == "softplus":
        model = nn.Sequential(model, nn.Softplus())
    else:
        assert config["head"] is None
    return model

# Training functions

In [7]:
def train_fn(model, optimizer, criterion, dl_train):

    loss_list = []
    model.train()
    for inpt, tgt in tqdm(dl_train):
        mb_size = inpt.shape[0]
        tgt = tgt.to_dense()

        optimizer.zero_grad()
        pred = model(inpt)

        loss = criterion(pred, tgt)
        loss_list.append(loss.detach())
        loss.backward()
        optimizer.step()
    avg_loss = sum(loss_list).cpu().item()/len(loss_list)
    
    return {"loss":avg_loss}


In [8]:
def valid_fn(model, criterion, dl_valid):
    loss_list = []
    all_preds = []
    all_tgts = []
    partial_correlation_scores = []
    model.eval()
    for inpt, tgt in tqdm(dl_valid):
        mb_size = inpt.shape[0]
        tgt = tgt.to_dense()
        with torch.no_grad():
            pred = model(inpt)
        loss = criterion(pred, tgt)
        loss_list.append(loss.detach())
        
        partial_correlation_scores.append(partial_correlation_score_torch_faster(tgt, pred))

    avg_loss = sum(loss_list).cpu().item()/len(loss_list)
    
    partial_correlation_scores = torch.cat(partial_correlation_scores)

    score = torch.sum(partial_correlation_scores).cpu().item()/len(partial_correlation_scores) #correlation_score_torch(all_tgts, all_preds)
    
    return {"loss":avg_loss, "score":score}


In [9]:
def train_model(model, optimizer, dl_train, dl_valid, save_prefix):

    criterion = config["criterion"]
    
    save_params_filename = save_prefix+"_best_params.pth"
    save_config_filename = save_prefix+"_config.pkl"
    best_score = None

    for epoch in range(config["max_epochs"]):
        log_train = train_fn(model, optimizer, criterion, dl_train)
        log_valid = valid_fn(model, criterion, dl_valid)

        print(log_train)
        print(log_valid)
        
        score = log_valid["score"]
        if best_score is None or score > best_score:
            best_score = score
            patience = config["patience"]
            best_params = copy.deepcopy(model.state_dict())
        else:
            patience -= 1
        
        if patience < 0:
            print("out of patience")
            break


    torch.save(best_params, save_params_filename)
    pickle.dump(config,open(save_config_filename, "wb"))
    


In [10]:
def train_one_fold(num_fold):
    
    train_idx, valid_idx = FOLDS_LIST[num_fold]
    
    train_idx = torch.from_numpy(train_idx).to(device)
    valid_idx = torch.from_numpy(valid_idx).to(device)
    
    
    dl_train = DataLoaderCOO(train_inputs, train_targets, train_idx=train_idx,
                batch_size=512, shuffle=True, drop_last=True)
    dl_valid = DataLoaderCOO(train_inputs, train_targets, train_idx=valid_idx,
                batch_size=512, shuffle=False, drop_last=False)
    
    model =  build_model()
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), **config["optimizerparams"])
    
    train_model(model, optimizer, dl_train, dl_valid, save_prefix="f%i"%num_fold)

# Load Data

In [11]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"machine has {torch.cuda.device_count()} cuda devices")
    print(f"model of first cuda device is {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")

machine has 1 cuda devices
model of first cuda device is Tesla P100-PCIE-16GB


In [12]:
%%time
train_inputs = scipy.sparse.load_npz(
    "../input/multimodal-single-cell-as-sparse-matrix/train_multi_inputs_values.sparse.npz")

CPU times: user 36.3 s, sys: 4.61 s, total: 40.9 s
Wall time: 1min 2s


We will normalize the input by dividing each column by its max value. This is the simplest reasonable option. Centering the data (i.e. substracting the mean, would destroy the sparsity here)

In [13]:
max_inputs = train_inputs.max(axis=0)
max_inputs = max_inputs.todense()+1e-10
np.savez("max_inputs.npz", max_inputs = max_inputs)
max_inputs = torch.from_numpy(max_inputs)[0].to(device)

In [14]:
%%time
train_inputs = load_csr_data_to_gpu(train_inputs)
gc.collect()

CPU times: user 1.17 s, sys: 25 ms, total: 1.19 s
Wall time: 1.19 s


51

In [15]:
train_inputs.data[...] /= max_inputs[train_inputs.indices.long()]

In [16]:
torch.max(train_inputs.data)

tensor(1., device='cuda:0')

In [17]:
%%time
train_targets = scipy.sparse.load_npz(
    "../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_values.sparse.npz")

CPU times: user 16.3 s, sys: 1.82 s, total: 18.1 s
Wall time: 26.5 s


In [18]:
%%time
train_targets = load_csr_data_to_gpu(train_targets)
gc.collect()

CPU times: user 893 ms, sys: 19 ms, total: 912 ms
Wall time: 928 ms


124

In [19]:
assert INPUT_SIZE == train_inputs.shape[1]
assert OUTPUT_SIZE == train_targets.shape[1]

NB_EXAMPLES = train_inputs.shape[0]
assert NB_EXAMPLES == train_targets.shape[0]

print(INPUT_SIZE, OUTPUT_SIZE, NB_EXAMPLES)

228942 23418 105942


# Training
We use a rather naive kfold split here, which might not be optimal for this competition.

In [20]:
kfold = KFold(n_splits=config["n_folds"], shuffle=True, random_state=config["kfold_random_state"])
FOLDS_LIST = list(kfold.split(range(train_inputs.shape[0])))

In [21]:
for num_fold in config["folds_to_train"]:
    train_one_fold(num_fold)

  0%|          | 0/165 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

{'loss': -0.6129105539032907}
{'loss': -0.6489819117954799, 'score': 0.6498714877117844}


  0%|          | 0/165 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

...

  0%|          | 0/42 [00:00<?, ?it/s]

{'loss': -0.6751399647105824}
{'loss': -0.6620621000017438, 'score': 0.662806814293468}


  0%|          | 0/165 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

{'loss': -0.6753891915986032}
{'loss': -0.662010011218843, 'score': 0.6627535799302672}
out of patience
